In [ ]:
#configurer et entrainer sur scRNAseq

c2l.models.RegressionModel.setup_anndata(
    adata=adata_sc,
    batch_key="features", #sample_id
    labels_key="cell_type",
    layer="nCount_SCT",
)

model = c2l.models.RegressionModel(adata_sc)
# default, try on GPU:
use_gpu = True
model.train(max_epochs=250, batch_size=2500, train_size=1, lr=0.002, use_gpu=use_gpu)




In [ ]:
#verification convergence

model.plot_history(20)

In [ ]:
# Exporter signatures

model.export_posterior(
    adata_sc,
    sample_kwargs={"num_samples": 1000, "batch_size": 2500, "use_gpu": use_gpu},
)

inf_aver = adata_sc.varm['means_per_cluster_mu_fg']  # genes x cell_types
inf_aver.to_csv('signatures.csv')  # réutilisable

In [ ]:
mod_ref.plot_QC()  # diagonal reconstruction

In [ ]:
#modele spatial (deconvolution)

c2l.models.Cell2location.setup_anndata(adata_st, batch_key='features')
mod_sp = c2l.models.Cell2location(adata_st, cell_state_df=inf_aver,
                                  N_cells_per_location=8)  # ~8 cellules/spot

mod_sp.train(max_epochs=30000)  # plus long, GPU recommandé
mod_sp.plot_history()  # convergence
mod_sp.plot_QC()  # diagonal

# Exporter abondances (q05 = confiance haute)
adata_st = mod_sp.export_posterior(adata_st, 
                                   sample_kwargs={'num_samples':1000})
adata_st.obs[mod_sp.factor_names_] = adata_st.obsm['q05_cell_abundance_w_sf']


In [ ]:
#visualization et validation

sc.pl.spatial(adata_st, color=['T-cells', 'Tumor'], 
              img_key='hires', vmin=0, vmax='p99')
sq.pl.spatial_scatter(adata_st, color='region_cluster')  # après clustering

# Clustering régions tissulaires
sc.pp.neighbors(adata_st, use_rep='q05_cell_abundance_w_sf')
sc.tl.leiden(adata_st)
sc.tl.umap(adata_st)

